# Top-selling model

A model that only recommend top selling products to customers

- How do we determine top selling products
- How does the top-selling model perform? 
- How many customers do actually purchase these products within a 7-day period?
- How many products are actually purchased?

Best selling items EDA on Kaggle: https://www.kaggle.com/negoto/best-selling-items-catalog-like-eda-of-articles

In [1]:
import pandas as pd
import numpy as np
import collaborative_filter as cf

In [2]:
%%time
transaction_raw = pd.read_csv("../data/transactions_train.csv")

CPU times: user 30.2 s, sys: 12.5 s, total: 42.7 s
Wall time: 50.8 s


In [3]:
#extract data between 2019-5-1 and 2019-6-1
mask = (transaction_raw.t_dat >= '2019-05-01') & (transaction_raw.t_dat <= '2019-06-01')
transaction_2019_May = transaction_raw.loc[mask].reset_index(drop=True)
transaction_2019_May

,t_dat,customer_id,article_id,price,sales_channel_id
0,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,524061003,0.050831,2
1,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,735404001,0.050831,2
2,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,700370004,0.016932,2
3,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,618800001,0.033881,2
4,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,731407001,0.016932,2
...,...,...,...,...,...
1622933,2019-06-01,fffdaab7d9d79ceee4ffd2ab98bb68411438ea90e46a67...,738881002,0.016932,2
1622934,2019-06-01,fffdaab7d9d79ceee4ffd2ab98bb68411438ea90e46a67...,220094010,0.025407,2
1622935,2019-06-01,ffff4c4e8b57b633c1ddf8fbd53db16b962cf831baf9ed...,487800001,0.011847,2
1622936,2019-06-01,ffff4c4e8b57b633c1ddf8fbd53db16b962cf831baf9ed...,754013001,0.022017,2


In [3]:
#save 2019 May data to csv
#transaction_2019_May.to_csv("../data/transactions_2019_May.csv", index=False)

In [4]:
#read 2019 May transaction data
transaction_2019_May = pd.read_csv("../data/transactions_2019_May.csv")
transaction_2019_May

,t_dat,customer_id,article_id,price,sales_channel_id
0,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,524061003,0.050831,2
1,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,735404001,0.050831,2
2,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,700370004,0.016932,2
3,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,618800001,0.033881,2
4,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,731407001,0.016932,2
...,...,...,...,...,...
1622933,2019-06-01,fffdaab7d9d79ceee4ffd2ab98bb68411438ea90e46a67...,738881002,0.016932,2
1622934,2019-06-01,fffdaab7d9d79ceee4ffd2ab98bb68411438ea90e46a67...,220094010,0.025407,2
1622935,2019-06-01,ffff4c4e8b57b633c1ddf8fbd53db16b962cf831baf9ed...,487800001,0.011847,2
1622936,2019-06-01,ffff4c4e8b57b633c1ddf8fbd53db16b962cf831baf9ed...,754013001,0.022017,2


In [4]:
#extract data between 2019-5-1 and 2019-5-10
mask = (transaction_2019_May.t_dat >= '2019-05-01') & (transaction_2019_May.t_dat <= '2019-05-10')
X_train = transaction_2019_May.loc[mask].reset_index(drop=True)
X_train

,t_dat,customer_id,article_id,price,sales_channel_id
0,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,524061003,0.050831,2
1,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,735404001,0.050831,2
2,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,700370004,0.016932,2
3,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,618800001,0.033881,2
4,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,731407001,0.016932,2
...,...,...,...,...,...
499329,2019-05-10,fffe7116f9f68e8ad287fd7b6e33aad4871d7080e77d2d...,609837003,0.027102,2
499330,2019-05-10,fffe7116f9f68e8ad287fd7b6e33aad4871d7080e77d2d...,638842003,0.049136,2
499331,2019-05-10,fffe7116f9f68e8ad287fd7b6e33aad4871d7080e77d2d...,510675001,0.027102,2
499332,2019-05-10,fffe7116f9f68e8ad287fd7b6e33aad4871d7080e77d2d...,771116006,0.067780,2


In [5]:
X_train = X_train[['customer_id','article_id']]
X_train

,customer_id,article_id
0,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,524061003
1,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,735404001
2,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,700370004
3,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,618800001
4,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,731407001
...,...,...
499329,fffe7116f9f68e8ad287fd7b6e33aad4871d7080e77d2d...,609837003
499330,fffe7116f9f68e8ad287fd7b6e33aad4871d7080e77d2d...,638842003
499331,fffe7116f9f68e8ad287fd7b6e33aad4871d7080e77d2d...,510675001
499332,fffe7116f9f68e8ad287fd7b6e33aad4871d7080e77d2d...,771116006


In [6]:
#check number of customers and items
print("Number of unique customer id:",X_train.head(100000).customer_id.nunique())
print("Number of unique article id:", X_train.head(100000).article_id.nunique())

Number of unique customer id: 24031
Number of unique article id: 15187


In [ ]:
%%time
recommend_items = cf.all_user_collaborative_filter(X_train.head(100000), similar_user_number = 100, max_recommend_items = 12)

In [11]:
mask = (transaction_2019_May.t_dat >= '2019-05-11') & (transaction_2019_May.t_dat <= '2019-05-17')
X_test = transaction_2019_May.loc[mask].reset_index(drop=True)
X_test = X_test[['customer_id','article_id']]
X_test

,customer_id,article_id
0,000416152a81973bebab23142bd5dd6ebd85c8c5e2d63e...,695544005
1,000416152a81973bebab23142bd5dd6ebd85c8c5e2d63e...,695632011
2,0006d3ff0caf0cb4d4e0615ee5cb7d268622364d483335...,748269002
3,00130faf36f2571cf7e08451b317545004a8a85327661f...,742191001
4,00130faf36f2571cf7e08451b317545004a8a85327661f...,347200001
...,...,...
331833,fff62d03f7f60e61a987a1b2c27a553dbae8a632df1322...,745475010
331834,fff62d03f7f60e61a987a1b2c27a553dbae8a632df1322...,708855002
331835,fff62d03f7f60e61a987a1b2c27a553dbae8a632df1322...,696128001
331836,fffd870c6324ad3bda24e4d6aeae221c199479086bfdfd...,700079006


In [30]:
MAP12_cf = cf.MAP(recommend_items, X_test)
print("User-based collaborative filter MAP@12:", MAP12_cf)

100%|███████████████████████████████████████████████████████████████████████████| 24031/24031 [01:18<00:00, 307.11it/s]

Number of users: 4332
User-based collaborative filter MAP@12: 0.0037136724997412925


In [31]:
##Identify top 12 selling items for training data
top12items = list(X_train.head(100000).article_id.value_counts()[0:12].index)

#save top 12 selling items for users in a dict
top12_items = {}
for user in recommend_items:
    top12_items[user] = top12items

In [33]:
MAP12_top = cf.MAP(top12_items, X_test)
print("Top 12 selling products MAP@12:", MAP12_top)

100%|███████████████████████████████████████████████████████████████████████████| 24031/24031 [01:21<00:00, 295.16it/s]

Number of users: 4332
Top 12 selling products MAP@12: 0.0020729474647988057


In [38]:
# % increase in MAP@12
print("MAP@12 increases by", round(100*(MAP12_cf-MAP12_top)/MAP12_top), "%")

MAP@12 increases by 79 %
